In [25]:
import os
import pandas as pd

In [26]:
DATASET_FOLDER = '../dataset/'
train = pd.read_csv(os.path.join(DATASET_FOLDER, 'train.csv'))
test = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))
sample_test = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test.csv'))
sample_test_out = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test_out.csv'))

In [1]:
import pandas as pd
df = pd.read_csv('../downloads/updated_image_ocr_results.csv')

C:\Users\y2ana\AppData\Local\Temp\ipykernel_16984\427502356.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../downloads/updated_image_ocr_results.csv')


In [295]:
test.shape

(131187, 4)

In [5]:
df = df.dropna()

In [6]:
df.sample()

,image_link,group_id,entity_name,entity_value,detected_text
20,https://m.media-amazon.com/images/I/31EvJszFVf...,731432,item_weight,200 gram,REE Glucon-0 9.4in 23cm


In [36]:
def findUnits(text):
    import re
    pattern = r"(\d+(?:\.\d+)?)\s*([A-Za-z]+)"
    matches = re.findall(pattern, text)
    return matches

In [98]:
n = 97
print("Text: ",df.iloc[n]['detected_text'])


Text:  BENEFITSOF GREEN COFFEE S`Herbal max Discover wellness 30 Helps Control Supports Blood Appetite* Sugar Level* Supports Weight Supports Immune managment* System* Boosts Herbal max Healthy Metabolism* Discover wellne Digestion* GREEN COFFEE BURNFAT Enhance Supports Mood* Fat Burn* Enhances Mood* Improves Digestion* 60 500MG Dietary Supplement CAPSULES


In [99]:
print("Image Link: ",df.iloc[n]['image_link'])

Image Link:  https://m.media-amazon.com/images/I/71uQmsTESvL.jpg


In [100]:
train[train['image_link']==df.iloc[n]['image_link']]

,image_link,group_id,entity_name,entity_value
97,https://m.media-amazon.com/images/I/71uQmsTESv...,459516,item_weight,500 milligram


In [102]:
df['detected_unit'] = df.detected_text.apply(lambda x:findUnits(x))


In [115]:
# all_units = []

# # Apply a lambda to append all units to the all_units list
# df.detected_unit.apply(lambda x: [all_units.append(y[1]) for y in x])

# # Print or process all_units
# print(all_units)


In [285]:
def extractedEntityValue(entity_name,unit_list):
    if(entity_name=='voltage'):
        # Use list comprehension to filter and find the values with 'v' (for volts)
        volt_list = [float(x[0]) for x in unit_list if x[1].lower() == 'v']
        
        # If volt_list is not empty, return the max value, else return a default value
        if volt_list:
            return f"{max(volt_list)} volt"
        else:
            return ""  # or an appropriate default value
    
    elif(entity_name=='wattage'):
        # Use list comprehension to filter and find the values with 'v' (for volts)
        volt_list = [float(x[0]) for x in unit_list if x[1].lower() == 'w']
        
        # If volt_list is not empty, return the max value, else return a default value
        if volt_list:
            return f"{max(volt_list)} watt"
        else:
            return ""  # or an appropriate default value
    
    elif entity_name == 'item_volume':
        # Filter values with 'ml', 'oz', or 'floz' for millilitres or ounces
        volume_list = [(float(x[0]), x[1].lower()) for x in unit_list if x[1].lower() in ['ml', 'oz', 'floz']]
        
        if volume_list:
            # Get the maximum value and its unit
            max_value, unit = max(volume_list)
            
            # Convert unit to full name
            if unit == 'ml':
                unit_name = 'millilitre'
            elif unit == 'floz':
                unit_name = 'fluid ounce'
            else:
                unit_name = 'ounce'
            
            return f"{max_value} {unit_name}"
        else:
            return ""
    
    elif entity_name in ['depth', 'width', 'height']:
        import random
        # Filter values with 'mm', 'cm', or 'm' for units of measurement
        depth_list = [(float(x[0]), x[1].lower()) for x in unit_list if x[1].lower() in ['mm', 'cm', 'm']]
        
        if depth_list:
            # Randomly select one of the measurements
            idx = random.randint(0, len(depth_list) - 1)  # Use randint to get a valid index
            max_value, unit = depth_list[idx]
            
            # Convert unit to full name
            if unit == 'mm':
                unit_name = 'millimeter'
            elif unit == 'cm':
                unit_name = 'centimeter'
            elif unit == 'm':
                unit_name = 'meter'
            
            return f"{max_value} {unit_name}"
        else:
            return ""

        

    elif entity_name == 'item_weight' or entity_name == 'maximum_weight_recommendation':
        # Weight conversion hashmap
        conversion_factors = {
            "mg": 1 / 1000,  # Convert mg to grams
            "g": 1,          # grams as base unit
            "kg": 1000,      # Convert kg to grams
            "oz": 28.34,     # Convert oz to grams
            "pound": 453.62,  # Convert pound to grams

        }

        # Handle various forms of grams
        def normalize_gram_unit(unit):
            unit = unit.lower()
            if unit in ['g', 'gm', 'gms','gpack','plantago','gsm','gn','grams']:
                return 'g'
            if unit in ['pound','lbs','']:
                return 'pound'
            if unit in ['kg','kgs']:
                return 'kg'
            
            return unit

        # List of converted weights in grams and original values
        weight_list = [(float(x[0]) * conversion_factors[normalize_gram_unit(x[1])], float(x[0]), normalize_gram_unit(x[1]))
                       for x in unit_list if normalize_gram_unit(x[1]) in conversion_factors]

        
        if weight_list:
            # Get the max weight in grams, along with the original value and unit
            _, max_value, unit = max(weight_list)
            
            # Mapping of unit abbreviations to full names
            unit_full_names = {
                "mg": "milligram",
                "g": "gram",
                "kg": "kilogram",
                "oz": "ounce",
                "pound": "pound"
            }

            unit_name = unit_full_names[unit]
            return f"{max_value} {unit_name}"
        else:
            return ""
    return ""


In [258]:
df['final_unit'] = df.apply(lambda x: extractedEntityValue(x['entity_name'],x['detected_unit']),axis=1)

In [289]:
import pandas as pd
import glob

# List all CSV files in a directory
csv_files = glob.glob('../downloads/parts_train/*.csv')

# Read and concatenate all CSV files
df = pd.concat((pd.read_csv(file) for file in csv_files), ignore_index=True)

# Print the combined DataFrame
print(df)


                                            image_link  group_id entity_name  \
0    https://m.media-amazon.com/images/I/51k7GMS8dg...    630869       depth   
1    https://m.media-amazon.com/images/I/51YfTCUszC...    630869       depth   
2    https://m.media-amazon.com/images/I/51F7vcA4YN...    630869       depth   
3    https://m.media-amazon.com/images/I/61EsldyUWp...    630869       depth   
4    https://m.media-amazon.com/images/I/51GzV0nG31...    675317       depth   
..                                                 ...       ...         ...   
395  https://m.media-amazon.com/images/I/6175zTg1bJ...    860821       width   
396  https://m.media-amazon.com/images/I/617f4K8Tlr...    844474       width   
397  https://m.media-amazon.com/images/I/518-uNTHNp...    648011       width   
398  https://m.media-amazon.com/images/I/610GTfdbRU...    752266       width   
399  https://m.media-amazon.com/images/I/614qXjuHQi...    801829       width   

         entity_value                  

In [292]:
df['detected_unit'] = df.detected_text.apply(lambda x:findUnits(x))
df['final_unit'] = df.apply(lambda x: extractedEntityValue(x['entity_name'],x['detected_unit']),axis=1)
df

,image_link,group_id,entity_name,entity_value,detected_text,detected_unit,final_unit
0,https://m.media-amazon.com/images/I/51k7GMS8dg...,630869,depth,21.0 centimetre,Enamel High polished Stainless steel 12mm Nylo...,"[(12, mm), (21, cm), (12.5, g)]",21.0 centimeter
1,https://m.media-amazon.com/images/I/51YfTCUszC...,630869,depth,20.5 centimetre,20.5cm .5mm IP black plating 47mm Stainless st...,"[(20.5, cm), (5, mm), (47, mm), (20.5, cm), (1...",20.5 centimeter
2,https://m.media-amazon.com/images/I/51F7vcA4YN...,630869,depth,63.0 millimetre,63mm IP gold plating 24mm Stainless steel High...,"[(63, mm), (24, mm), (15.5, cm), (4, mm), (8, g)]",63.0 millimeter
3,https://m.media-amazon.com/images/I/61EsldyUWp...,630869,depth,5.0 centimetre,PRODUCT PARAMETERS Rhodium Plated Name Ca Pe...,"[(5, cm), (2.9, g)]",5.0 centimeter
4,https://m.media-amazon.com/images/I/51GzV0nG31...,675317,depth,132.0 millimetre,L:132mm Hole Distance: 64mm 12mm H: 16mm 58mm ...,"[(132, mm), (64, mm), (12, mm), (16, mm), (58,...",16.0 millimeter
...,...,...,...,...,...,...,...
395,https://m.media-amazon.com/images/I/6175zTg1bJ...,860821,width,380.0 millimetre,380MM 300MM INDOOR AIR OUTDOOR AIR CIRCULATION...,"[(380, MM), (300, MM), (500, MM), (100, MM), (...",300.0 millimeter
396,https://m.media-amazon.com/images/I/617f4K8Tlr...,844474,width,24.0 centimetre,SSSKYSPER Live with Passion LIGHTWEIGHT MODEL ...,"[(1.73, m), (8, Weight), (0.62, lb), (285, g),...",19.0 centimeter
397,https://m.media-amazon.com/images/I/518-uNTHNp...,648011,width,8.0 centimetre,8cm/3.15in 14cm/5.51in,"[(8, cm), (3.15, in), (14, cm), (5.51, in)]",14.0 centimeter
398,https://m.media-amazon.com/images/I/610GTfdbRU...,752266,width,285.0 millimetre,PANELMOD.ROMADOWNLIGHT REDONDO 25WEMPOTRABLE P...,"[(25, WEMPOTRABLE), (25, W), (25, Medidas), (2...",300.0 millimeter


In [294]:
(df['final_unit']==df['entity_value']).sum()/400

0.285

entity_name
height                           32282
depth                            28146
width                            26931
item_weight                      22032
maximum_weight_recommendation     7028
voltage                           5488
wattage                           5447
item_volume                       3833
Name: count, dtype: int64

In [262]:
weight_df = pd.read_csv('../downloads/parts_train/item_weight_ocr_results.csv')
weight_df['detected_unit'] = weight_df.detected_text.apply(lambda x:findUnits(x))
weight_df['final_unit']=weight_df.apply(lambda x: extractedEntityValue(x['entity_name'],x['detected_unit']),axis=1)
weight_df[weight_df['entity_value']!=weight_df['final_unit']].sample(4)

,image_link,group_id,entity_name,entity_value,detected_text,detected_unit,final_unit
49,https://m.media-amazon.com/images/I/71E9iF-bmK...,731432,item_weight,1 kilogram,DESIGNED FOR EXTREME GAINS Support Weight & Mu...,"[(1012, g), (200, g), (12, g)]",1012.0 gram
48,https://m.media-amazon.com/images/I/71ldprwbKr...,731432,item_weight,10 kilogram,PREMIUM OUALITY NGRIDIENTS ADDED HIGH QUALITY ...,"[(60, g), (12, g)]",60.0 gram
5,https://m.media-amazon.com/images/I/81xsq6vf2q...,731432,item_weight,1400 milligram,"Horbaach Directions: For adults, take two (2) ...","[(1400, mg), (220, ml), (1400, MG), (1.5, hour...",1400.0 milligram
26,https://m.media-amazon.com/images/I/613v+2W4Uw...,524635,item_weight,18.55 gram,"50 mI Carvomin R Verdauungstropfen 18,55 g/20 ...","[(50, mI), (55, g), (20, ml)]",55.0 gram


In [263]:

weight_df = pd.read_csv('../downloads/parts_train/voltage_ocr_results.csv')
weight_df['detected_unit'] = weight_df.detected_text.apply(lambda x:findUnits(x))
weight_df['final_unit']=weight_df.apply(lambda x: extractedEntityValue(x['entity_name'],x['detected_unit']),axis=1)
weight_df[weight_df['entity_value']!=weight_df['final_unit']].sample(4)

,image_link,group_id,entity_name,entity_value,detected_text,detected_unit,final_unit
42,https://m.media-amazon.com/images/I/71OEmk050T...,179080,voltage,48.0 volt,CTYELECTRIC BIKE 250W48VM0T0R GEAREDMOTOR 25KM...,"[(250, W), (48, VM), (0, T), (0, R), (25, KM),...",
14,https://m.media-amazon.com/images/I/51xK4g4F03...,271537,voltage,240.0 volt,Error: 'NoneType' object is not iterable,[],
49,https://m.media-amazon.com/images/I/7195HI0YTA...,752266,voltage,"[110.0, 240.0] volt",10 cm E27 MAX 60W (BULB NOT INCLUDED) Chain ad...,"[(10, cm), (27, MAX), (60, W), (240, V), (60, ...",240.0 volt
5,https://m.media-amazon.com/images/I/71Ye12U8zR...,271537,voltage,12.0 volt,31ft 2V,"[(31, ft), (2, V)]",2.0 volt


In [264]:

weight_df = pd.read_csv('../downloads/parts_train/wattage_ocr_results.csv')
weight_df['detected_unit'] = weight_df.detected_text.apply(lambda x:findUnits(x))
weight_df['final_unit']=weight_df.apply(lambda x: extractedEntityValue(x['entity_name'],x['detected_unit']),axis=1)
weight_df[weight_df['entity_value']!=weight_df['final_unit']].sample(4)

,image_link,group_id,entity_name,entity_value,detected_text,detected_unit,final_unit
15,https://m.media-amazon.com/images/I/51WaQdTFyh...,955292,wattage,3.0 watt,capslocke shiftt Data Transfer up to 5Gbps,"[(5, Gbps)]",
34,https://m.media-amazon.com/images/I/61K8Ua5gvk...,276700,wattage,1800.0 watt,IPRECAUCION IATTENTION PWR JUMP PWRCORE12 LITH...,"[(12, LITHIUM)]",
17,https://m.media-amazon.com/images/I/818ZG7g9hH...,746096,wattage,21.6 watt,Product Information Sheet. COMMISSION DELEGATE...,"[(2013, with), (1935, Za), (27, M), (6, w), (1...",6.0 watt
39,https://m.media-amazon.com/images/I/31UmFlrQMD...,752266,wattage,2.4 watt,LIGHTING SAFE LED 6000,[],


In [265]:

weight_df = pd.read_csv('../downloads/parts_train/item_volume_ocr_results.csv')
weight_df['detected_unit'] = weight_df.detected_text.apply(lambda x:findUnits(x))
weight_df['final_unit']=weight_df.apply(lambda x: extractedEntityValue(x['entity_name'],x['detected_unit']),axis=1)
weight_df[weight_df['entity_value']!=weight_df['final_unit']].sample(4)

,image_link,group_id,entity_name,entity_value,detected_text,detected_unit,final_unit
37,https://m.media-amazon.com/images/I/91+No1yJ4t...,281678,item_volume,"[70.0, 100.0] millilitre",H Premium Quality 3C OC HERBALTEA 3C Hibiscus ...,"[(3, C), (3, C), (0, XHXHH)]",
0,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup,TEARRIFIC LEBENSMITTELECHT Cup GEPRAGTES DESIG...,[],
14,https://m.media-amazon.com/images/I/71pKB7-3it...,459516,item_volume,16.0 fluid ounce,FOR BEST RESULT COMB NE Earth's Creation Earth...,"[(1, OOOmg), (3, in), (0, Supports), (60, Caps...",
29,https://m.media-amazon.com/images/I/81OYyqNGOm...,373285,item_volume,8.0 fluid ounce,100% JUICE INGREDIENTS: STRAWBERRY PUREE. APPL...,"[(8, servings), (8, fl), (240, mL), (120, Calo...",240.0 millilitre


In [267]:
(weight_df['entity_value']!=weight_df['final_unit']).sum()

27

In [278]:
depth_df = pd.read_csv('../downloads/parts_train/depth_ocr_results.csv')
depth_df['detected_unit'] = depth_df.detected_text.apply(lambda x:findUnits(x))
depth_df.sample()

,image_link,group_id,entity_name,entity_value,detected_text,detected_unit
44,https://m.media-amazon.com/images/I/51eMhn9AbU...,630869,depth,21.0 centimetre,Tiger Eye Total Length of Tigereye bracelet:21...,"[(21, cm), (1.2, cm), (2.7, cm), (24.8, g)]"


In [283]:
depth_df = pd.read_csv('../downloads/parts_train/maximum_weight_recommendation_ocr_results.csv')
depth_df['detected_unit'] = depth_df.detected_text.apply(lambda x:findUnits(x))


In [284]:
depth_df

,image_link,group_id,entity_name,entity_value,detected_text,detected_unit
0,https://m.media-amazon.com/images/I/71d+dz7ogk...,150913,maximum_weight_recommendation,15 kilogram,AGE GROUP 0-36M WEIGHT CAPACITY 15 KGS ist Step,"[(36, M), (15, KGS)]"
1,https://m.media-amazon.com/images/I/71k0T1lkaz...,801829,maximum_weight_recommendation,350 pound,MAISONARTS 350 LBS 20.55LBS PRODUCT CAPACITY 3...,"[(350, LBS), (20.55, LBS)]"
2,https://m.media-amazon.com/images/I/81OX3Lrynv...,801829,maximum_weight_recommendation,120 kilogram,Suitable for Various Occasions welcome Bedroom...,"[(120, kg), (120, kg)]"
3,https://m.media-amazon.com/images/I/71kKyflqh-...,178958,maximum_weight_recommendation,7 kilogram,LIVINGBASICS Serves All Your Storage Needs 222...,"[(2222222, IN), (7, Required), (7, Capacity), ..."
4,https://m.media-amazon.com/images/I/71Yl8r0WgH...,120219,maximum_weight_recommendation,12.0 ounce,Foldable design. Convenient storage saves spac...,[]
5,https://m.media-amazon.com/images/I/81xqMNmhEp...,446789,maximum_weight_recommendation,300 kilogram,5 YEARS 300 KGS ARRAN 5 Years Warranty Weight ...,"[(5, YEARS), (300, KGS), (5, Years), (2, Adults)]"
6,https://m.media-amazon.com/images/I/7104ARm7I-...,801829,maximum_weight_recommendation,440.0 pound,"PRODUCT SIZE 37.4' 26.0"" 45.0"" 440 Ibs 20"" Wei...","[(440, Ibs), (18.7, Ibs)]"
7,https://m.media-amazon.com/images/I/61L4wjI4YE...,120219,maximum_weight_recommendation,6.0 ton,8 5 6 D,"[(6, D)]"
8,https://m.media-amazon.com/images/I/71ubk42imP...,969033,maximum_weight_recommendation,15 pound,Please Check Dog Size Before The Purchasing Si...,"[(60, x), (45, x), (20, CM), (15, LBS), (75, x..."
9,https://m.media-amazon.com/images/I/7127o10t9t...,801829,maximum_weight_recommendation,250 pound,Weight Capacity 250 LBS 23.6in 28.7in 19.6in 1...,"[(250, LBS), (23.6, in), (28.7, in), (19.6, in..."
